
# Interface to C++

- With Julia having been built on llvm, Keno Fischer hooked into libclang to compile C++ expressions and to unravel C++ types and methods.
- The resulting package, `Cxx.jl`, allows for evaluating C++ code from within Julia.

In [7]:
using Cxx

## Reading feather files using C++ in Julia

- The [feather](https://github.com/wesm/feather) file format provides serialized storage of column-oriented data frames.
- The metadata is stored in the file in a schema that uses Google's FlatBuffers.
- Google's `flatc` can produce deserialization code in several possible languages from a schema description.
- The `native` implementation is in `C++`.  `flatc` produces a header file that depends only on `flatbuffers.h`

In [8]:
; tree /var/tmp/Feather

/var/tmp/Feather
├── appveyor.yml
├── deps
│   └── usr
│       └── include
│           ├── feather
│           │   ├── metadata.fbs
│           │   └── metadata_generated.h
│           └── flatbuffers
│               └── flatbuffers.h
├── LICENSE.md
├── README.md
├── REQUIRE
├── src
│   ├── column.jl
│   ├── Feather.jl
│   └── reader.jl
└── test
    ├── data
    │   ├── airquality.feather
    │   ├── anscombe.feather
    │   ├── attenu.feather
    │   ├── attitude.feather
    │   ├── beaver1.feather
    │   ├── beaver2.feather
    │   ├── BOD.feather
    │   ├── cars.feather
    │   ├── ChickWeight.feather
    │   ├── chickwts.feather
    │   ├── CO2.feather
    │   ├── DNase.feather
    │   ├── esoph.feather
    │   ├── faithful.feather
    │   ├── Formaldehyde.feather
    │   ├── freeny.feather
    │   ├── Indometh.feather
    │   ├── infert.feather
    │   ├── InsectSprays.feather
    │   ├── iris.feather
    │   ├── LifeCycleSavings.feather
    │   ├── Loblolly.feather
    │   ├── 

In [9]:
include("/var/tmp/Feather/src/Feather.jl")

Feather

In [10]:
rd = Feather.Reader("/tmp/br.feather")

[10001 × 2] @ /tmp/br.feather
 br1  : Array{Float64,1}
 br2  : Array{Float64,1}


  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/bates/.julia/v0.5/IJulia/src/execute_request.jl


In [5]:
using DataFrames
df = DataFrame(rd)

  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/bates/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/bates/.julia/v0.5/IJulia/src/execute_request.jl


,br1,br2
1,0.0,0.0
2,-0.7738342877376405,-0.006074935672474525
3,-0.5375807147247686,-0.2811773659446651
4,0.0702390093222015,0.989619761949976
5,0.9139219957343697,0.27246147956367606
6,0.7832533923397063,0.9439573263975158
7,1.847160721316817,0.7960476452392949
8,0.44813219966871465,1.4786680546016089
9,-2.3587306546636904,0.10640892780049693
10,-3.037558666825776,-0.3222091562244578


```jl
module Feather

using Cxx, DataArrays, DataFrames
import DataFrames: DataFrame, names, ncol, nrow

using Cxx: CppPtr, CxxQualType, CppBaseType

addHeaderDir(joinpath(dirname(@__FILE__), "..", "deps", "usr", "include"))
cxxinclude(  joinpath(dirname(@__FILE__), "..", "deps", "usr", "include", "feather", "metadata_generated.h"))

typealias CTablePt CppPtr{CxxQualType{CppBaseType{Symbol("feather::fbs::CTable")},(true,false,false)},(false,false,false)}
typealias ColumnPt CppPtr{CxxQualType{CppBaseType{Symbol("feather::fbs::Column")},(true,false,false)},(false,false,false)}
typealias PrimitivePt CppPtr{CxxQualType{CppBaseType{Symbol("feather::fbs::PrimitiveArray")},(true,false,false)},(false,false,false)}
typealias CategoryMetadataPt CppPtr{CxxQualType{CppBaseType{Symbol("feather::fbs::CategoryMetadata")},(true,false,false)},(false,false,false)}

export
    DataFrame,
    names,
    ncol,
    nrow

const magic = "FEA1"

include("column.jl")
include("reader.jl")

end
```

In [11]:
@less Feather.Reader("Foo")

type Reader
    path::AbstractString
    tpt::CTablePt
    buf::IO
    columns::Vector{Column}
end

columns(ct::CTablePt) = icxx"$ct->columns();"
description(ct::CTablePt) = unsafe_wrap(String, icxx"$ct->description();")
DataFrames.nrow(ct::CTablePt) = icxx"$ct->num_rows();"
version(ct::CTablePt) = icxx"$ct->version();"


function Reader(path::AbstractString)
    io = IOBuffer(Mmap.mmap(path))
    bpt = pointer(io.data)
    pos = position(skip(seekend(io), -8)) # last 8 bytes are [..., UInt32(metadata size), magic]
    pos -= read(io, Int32)                # start of the metadata
                                          # file should begin and end with "FEA1"
    if io.data[1:4] ≠ magic.data || read(io) ≠ magic.data
        throw(ArgumentError(string("File: ", path, " is not in feather format")))
    end
    tpt = icxx"feather::fbs::GetCTable($(pointer(io.data) + pos));"
    cpt = columns(tpt)
    cols = [Column(cpt, i, bpt) for i in 1:icxx"$cpt->size();"]
    Reader(path, tpt, io, co